## Q-1. Data collection

## 1.1. Get the list of books

In [2]:
from bs4 import BeautifulSoup as soup
import requests
import time
import urllib.request
import urllib.error as uer
from pathlib import Path
from lxml import etree, html
import regex as re
import csv
import pandas as pd
from langdetect import detect
from tqdm import tqdm
import nltk
import io 
import codecs
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import stem
from string import punctuation
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
from collections import Counter
import heapq
import math

In [ ]:
#taking all web pages of books' rank
#https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1

url_rank = []
for i in tqdm(range(1,301)):
    url_rank.append('https://www.goodreads.com/list/show/1.Best_Books_Ever?page={}'.format(i))

string_for_parsing = 'https://www.goodreads.com' #url of the site

link_to_download = []
for num_page in tqdm(url_rank):
    book_prova  = requests.get(num_page)
    page = soup(book_prova.text, 'html.parser')
    domenica = page.find_all('a', {'class' : 'bookTitle'})
    for item in domenica:
        link_to_download.append(string_for_parsing+item['href'])

#write txt file 
with open('urls.txt', 'w') as f:
    for item in tqdm(link_to_download):
        f.write("%s\n" % item)

## 1.2. Crawl books

In [5]:
Path("directory").mkdir(exist_ok=True)

for i in range(1,4):
    nomeCartella = 'cartella{}'.format(i)
    Path(nomeCartella).mkdir(exist_ok=True)
    
def download_html():
    id = int(input())  #we split download in 3 parts to work in parallel
                        #Luca = 1  #Mert = 2  #Nello = 3
    if id ==1:
        a = 0
        b = 10000
        i = 0
    elif id ==2:
        a = 11900
        b = 20000
        i = 99
    elif id == 3:
        a = 20000
        b = 30000
        i = 199
    
    for url in range(a,b):
        
        if url == 0 or url%100 == 0:
            i += 1
            folder_name = 'html_books_{}'.format(i)
            Path(folder_name).mkdir(exist_ok=True)
        urllib.request.urlretrieve(link_to_download[url], 'html_books_{}/article_{}.html'.format(i,url+1))
        
download_html()

## 1.3 Parse downloaded pages

In [19]:
def tsv_writer():
    
    print('Insert Path ')
    
    path = input()
    
    print('Insert directory')
    
    directory = input()
    
    count_error = []
    
    for j in range(0,201):
        a = 1 + 100*(j)
        b = 101 + 100*(j)
        for k in range(a,b):
            
            try:

                HtmlFile = open(path+'/'+directory+str(j)+'/article_{}.html'.format(k), 'r', encoding='utf-8')

                source_code = HtmlFile.read()

                book = soup(source_code, 'html.parser')

                book_title = book.find('h1',{'id':'bookTitle'}).contents[0].strip()

                book_series = book.find('h2',{'id':'bookSeries'})

                if book_series.text.strip() == '':
                    series = ''
                else:
                    series = book_series.contents[1].contents[0].strip()

                author = book.find('span',{'itemprop':'name'}).contents[0]

                rating = float(book.find('span',{'itemprop':'ratingValue'}).contents[0].strip())

                number_of_rating = int(book.find('meta',{'itemprop':'ratingCount'}).contents[0].split()[0].replace(',',''))

                review_count = int(book.find('meta',{'itemprop':'reviewCount'})['content'])

                number_of_pages = int(book.find('span',{'itemprop':'numberOfPages'}).contents[0].split()[0])

                publishing_date = book.find('div',{'class':'row'}).find_next_sibling().contents[0].split()[1:4]

                plot = book.find('div',{'id':'description'})
                
                try:                
                
                    complete_plot = plot.find_all('span')
                    
                    if detect(str(complete_plot)) != 'en':
                        continue
                
                except: 
                    
                    complete_plot = ''
                    

                details = book.find('div',{'id':'bookDataBox'})

                places = details.find_all('a', {'href': re.compile(r'/places')})

                settings = ''
                for item in places:
                    settings += item.text+','


                character_sw = details.find_all('a', {'href': re.compile(r'/characters')})

                character = ''
                for item in character_sw:
                    character+= item.text+','
            
            except:
                
                count_error.append((a,j))
                
                continue


            with open('books_tsv/article_{}.tsv'.format(k), 'wt', encoding="utf-8") as out_file:
                    tsv_writer = csv.writer(out_file, delimiter='\t')
                    tsv_writer.writerow(['book_title','book_series','author','rating','number_of_rating',
                                         'review_count','number_of_pages','publishing_date','complete_plot','places','characters'])
                    tsv_writer.writerow([book_title,series,author,rating,number_of_rating,review_count,number_of_pages,
                                        publishing_date,complete_plot,settings,character])


In [20]:
tsv_writer()

Insert Path 
books
Insert directory
html_books_


## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the books that match the query.

First, we have pre-processed all the information collected for each book by

* Removing stopwords
* Removing punctuation
* Stemming

### 2.1. Conjunctive query

#### 2.1.1) Create your index!

Before building the index,

We have created a file named vocabulary, in the .csv format, that maps each word to an integer (term_id).
Then, the first brick of the homework is to create the Inverted Index. It will be a dictionary of this format:

{

term_id_1:[document_1, document_2, document_4],

term_id_2:[document_1, document_3, document_5, document_6],

...}

where document_i is the id of a document that contains the word.

In [ ]:
def cleaner(text):  #this functions does the preprocessing work on the string given as input and returns the cleaned string

    words = re.split(r'\W+', str(text))
    words = [word.lower() for word in words]
    without_punct = [wp for wp in words if wp not in punctuation]
    sw = stopwords.words('english')
    without_sw = [w for w in without_punct if w not in sw] 
    list_to_remove = ['b','br','span', 'one' , 'id', 'none' ]
    clean_more = [w for w in without_sw if w not in list_to_remove] 
    clean_more_2 = [w for w in clean_more if  not (re.findall(re.compile(r'freetext'),w)
                                                   or re.findall(re.compile(r'\d'),w)) ]
    
    
    ps = nltk.stem.PorterStemmer()
    stemmed_list=[ps.stem(w) for w in clean_more_2 ]
    return ' '.join(stemmed_list)

for books in range(20001,30001):
    text_1=''
    try:
        df = pd.read_csv("books_3_tsv/article_"+str(books)+".tsv", sep='\t', encoding='utf-8' ) #vocab read csv files
    except:
        continue
    df = df.fillna("")
    df['book_title']=df['book_title'].apply(cleaner)
    df['complete_plot']=df['complete_plot'].apply(cleaner)
    #df['book_series']=df['book_series'].apply(cleaner)

    #df.to_csv('cleaned_csv/prova_libro_'+str(books)+'.csv') #to create new cleaned csv use this line 
    
    #now create the vocab
    
    title = df.iloc[0, 0]
    plot = df.iloc[0, 8]
    text_1 = title + " " + plot
    arr_1=(text_1.split())
    c = Counter(arr_1)  # create a dictionary with the number of occurences for each word
    file = open("txt_files_2/article_"+str(books)+".txt", "w")  # save it in a txt files 
    file.write(str(c)) 
    file.close() 

#save vocabulary as csv with indices for each vocab
arr = set(text.split()) # use set to eliminate repeating words
vocab = pd.DataFrame(arr)
vocab.to_csv("vocab.csv") #save vocabulary as csv with indices for each vocab

df_voc= pd.read_csv('vocab.csv')
df_voc.columns=['id','word']

from collections import defaultdict
d = defaultdict(set)
import ast

def extract_words_2(id):
    try:        
        file = open("txt_files/article_"+str(doc)+ ".txt", "r") 
        test=ast.literal_eval(file.read())
        file.close()
        return test
    except:
        return []




## Q-3. Define a new score!

As it has been requested we have added a new filter. We are asking to user to enter the query and the minimum rating for the books. We have defined a new score which is taking the rating into account and normalize it then multiply by cosine similarity. Then we are showing the 10 books with the highest new score that has been defined.

In [12]:
def cleaner(text):

    words = re.split(r'\W+', str(text))
    words = [word.lower() for word in words]
    without_punct = [wp for wp in words if wp not in punctuation]
    sw = stopwords.words('english')
    without_sw = [w for w in without_punct if w not in sw] 
    list_to_remove = ['b','br','span', 'one' , 'id', 'none' ]
    clean_more = [w for w in without_sw if w not in list_to_remove] 
    clean_more_2 = [w for w in clean_more if  not (re.findall(re.compile(r'freetext'),w)
                                                   or re.findall(re.compile(r'\d'),w)) ]
    
    
    ps = nltk.stem.PorterStemmer()
    stemmed_list=[ps.stem(w) for w in clean_more_2 ]
    return ' '.join(stemmed_list)

In [19]:
def find_link(index):
    ind=str(index)
    
    folder = math.floor((index-1)/100)    
    
    
    HtmlFile = open('books/html_books_'+str(folder)+'/article_{}.html'.format(ind), 'r', encoding='utf-8')
    source_code = HtmlFile.read()

    book = soup(source_code, 'html.parser')

    book_link = book.find('link')
    
    return book_link['href']

In [21]:
# Create a csv files for all books with cleaned book title, rating, cleaned plot, url, empty similarity and score column.
# The similarity and new score that has been defined will be added to dataframe later.

df_new = pd.DataFrame(columns = ['Book Title','Rating', 'Number_of_Rating', 'Plot', 'Url', 'Similarity', 'Score'])
title_list = []
plot_list = []
url_list = []
rating_list = []
num_rating_list = []

for index in tqdm(range(1,30)):
    try:
        df = pd.read_csv("books_tsv/article_"+str(index)+".tsv", sep='\t', encoding='utf-8' )
    except:
        continue
    df['book_title'] = df['book_title'].apply(cleaner)
    title = df['book_title'].iloc[0]
    df['complete_plot']=df['complete_plot'].apply(cleaner)
    plot = df['complete_plot'].iloc[0]
    complete_plt= title + " " + plot
    plot_split = re.split(r'\W+', str(complete_plt))
    rate = df['rating'].iloc[0]/5 # Normalize the rating
    num_rating = df['number_of_rating'].iloc[0]
    
    url=find_link(index)    
    title_list.append(title)
    plot_list.append(' '.join(plot_split))
    url_list.append(url)
    rating_list.append(rate)
    num_rating_list.append(num_rating)
    
df_new['Book Title'] = title_list
df_new['Plot'] = plot_list
df_new['Url'] = url_list
df_new['Rating'] = rating_list
df_new['Number_of_Rating'] = num_rating_list
df_new.to_csv ('tfidf_cos2.csv', index = False, header=True)

df_new = pd.read_csv("tfidf_cos2.csv")

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.40it/s]


In [22]:
# Calculate the tfidf and cosine similarity score for all books with the query

vectorizer = TfidfVectorizer() # Get tf-idf matrix using fit_transform function
X = vectorizer.fit_transform(df_new['Plot'].values.astype('U')) # Store tf-idf representations of all docs

query = str(input('Search something: ')) # Ask the user to write the query
min_rating = np.float(input('Minimum rating: ')) # We have asked to user to write the minimum rating he/she wants
min_number_rating = int(input('Minimum number of rating: ')) # We have asked to user to write the minimum rating he/she wants

query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
results = cosine_similarity(X,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
cos_sim = results.tolist() # Convert the cosine similarity result to list format
df_new['Similarity'] = cos_sim
df_new['Score'] = cos_sim * df_new['Rating']  # Calculate the score as cosine_similarity*rating and write to Score column of dataframe

df_empty = pd.DataFrame(columns = ['Book Title','Rating', 'Plot', 'Url', 'Similarity','Score'])

for i in tqdm(range(len(df_new))):
    if df_new.loc[i]['Rating']*5 >= min_rating and df_new.loc[i]['Number_of_Rating'] >= min_number_rating: # Filter the books with minimum rating provided by user.
        df_updated = pd.concat([df_empty, df_new], axis=0) # Add the book to updated dataframe which has rating greater than min rating

df_updated.nlargest(10,'Score') # Take the first 10 books with larger new score

Search something: last freedom
Minimum rating: 3.5
Minimum number of rating: 10000


100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 427.96it/s]


,Book Title,Rating,Plot,Url,Similarity,Score,Number_of_Rating
11,hitchhik guid galaxi,0.844,hitchhik guid galaxi second earth demolish mak...,https://www.goodreads.com/book/show/386162.The...,0.100256,0.084616,1446532.0
22,lord fli,0.738,lord fli dawn next world war plane crash uncha...,https://www.goodreads.com/book/show/7624.Lord_...,0.093382,0.068916,2274444.0
6,anim farm,0.790,anim farm librarian note altern cover edit edi...,https://www.goodreads.com/book/show/170448.Ani...,0.038673,0.030552,2759761.0
0,hunger game,0.866,hunger game could surviv wild everi make sure ...,https://www.goodreads.com/book/show/2767052-th...,0.000000,0.000000,6411305.0
1,harri potter order phoenix,0.900,harri potter order phoenix door end silent cor...,https://www.goodreads.com/book/show/2.Harry_Po...,0.000000,0.000000,2526182.0
2,kill mockingbird,0.856,kill mockingbird unforgett novel childhood sle...,https://www.goodreads.com/book/show/2657.To_Ki...,0.000000,0.000000,4529336.0
3,pride prejudic,0.852,pride prejudic altern cover edit isbn sinc imm...,https://www.goodreads.com/book/show/1885.Pride...,0.000000,0.000000,3019217.0
4,twilight,0.720,twilight three thing absolut posit first edwar...,https://www.goodreads.com/book/show/41865.Twil...,0.000000,0.000000,4991938.0
5,book thief,0.874,book thief librarian note altern cover edit fo...,https://www.goodreads.com/book/show/19063.The_...,0.000000,0.000000,1846649.0
7,chronicl narnia,0.852,chronicl narnia journey end world fantast crea...,https://www.goodreads.com/book/show/11127.The_...,0.000000,0.000000,522051.0


0
3
6
9
12
15
18
